- Carga el CSV en Spark.

In [ ]:
from pyspark.sql import SparkSession

# Create or get a Spark session
spark = SparkSession.builder.appName("Read Compressed CSV from S3").getOrCreate()

# Define the path to your S3 bucket and compressed CSV files
s3_bucket_path = "s3://mdge-e3-2024/*.csv.gz"

# Read the compressed CSV files into a DataFrame
df = spark.read.csv(s3_bucket_path, header=True, inferSchema=True)

In [5]:
# Add "year" and "month" columns based on "fecha_registro"
from pyspark.sql import functions as F
df = df.withColumn("year", F.year("fecha_registro"))
df = df.withColumn("month", F.month("fecha_registro"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- Guarda el CSV como parquet en S3, particionalo por `catalogo`. (Utiliza todos los trucos que consideres).

In [7]:
# Define output directory
output_directory = "s3://mdge-e3-2024/parquet_files_partitioned/"

# Write the DataFrame to Parquet, partitioned by 'categoria', 'year', and 'month'
df.write.partitionBy("categoria", "year", "month").mode('overwrite').parquet(output_directory)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- Carga el parquet en Spark.

In [8]:
# Read the Parquet files into a DataFrame
df_parquet = spark.read.parquet(output_directory)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Contesta las siguientes preguntas utilizando PySpark. Realiza el siguiente análisis **(por año)** y sobre todos los catálogos.

- ¿Cuántos catálogos diferentes tenemos?

In [9]:
from pyspark.sql.functions import countDistinct

# Count distinct catalogs
distinct_catalogs = df_parquet.agg(countDistinct("catalogo").alias("distinct_catalogs"))
distinct_catalogs.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|distinct_catalogs|
+-----------------+
|               12|
+-----------------+

- ¿Cuáles son los20 catálogos con más observaciones? Guarda la salida de este query en tu bucket de S3, lo necesitaremos más adelante.

In [11]:
from pyspark.sql.functions import desc

# Find top 20 catalogs by number of observations
top_20_catalogs = df_parquet.groupBy("catalogo").count().orderBy(desc("count")).limit(20)
top_20_catalogs.show()

# Save the output to S3
top_20_catalogs.write.mode('overwrite').parquet("s3://mdge-e3-2024/top_20_catalogs/")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+
|           catalogo|   count|
+-------------------+--------+
|            basicos|72474782|
|       medicamentos|29402008|
|  electrodomesticos|12276099|
| frutas y legumbres| 7571260|
|   utiles escolares| 5160328|
|           mercados| 3292185|
|           juguetes| 2651525|
|              pacic| 1079162|
|pescados y mariscos|  789438|
|          navidenos|  428681|
|              tenis|   31626|
|        aeropuertos|     581|
+-------------------+--------+

- ¿Tenemos datos de todos los estados del país? De no ser así, ¿cuáles faltan?

In [ ]:
all_states = [
    "aguascalientes", "baja california", "baja california sur", "campeche", "chiapas",
    "chihuahua", "ciudad de mexico", "distrito federal", "coahuila de zaragoza", "colima", "durango", "guanajuato",
    "guerrero", "hidalgo", "jalisco", "estado de mexico", "michoacan de ocampo", "morelos", "nayarit",
    "nuevo leon", "oaxaca", "puebla", "queretaro", "quintana roo", "san luis potosi",
    "sinaloa", "sonora", "tabasco", "tamaulipas", "tlaxcala", "veracruz", "yucatan", "zacatecas"
]

# Get distinct states from DataFrame
states_in_data = df_parquet.select("estado").distinct().rdd.flatMap(lambda x: x).collect()

# Find missing states
missing_states = [state for state in all_states if state not in states_in_data]
print("Missing States:", missing_states)

- ¿Cuántas observaciones tenemos por estado?

In [13]:
# Count observations by state
observations_by_state = df_parquet.groupBy("estado").count()
observations_by_state.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------+
|          estado|   count|
+----------------+--------+
|      tamaulipas| 3836149|
|       zacatecas| 3221774|
|      nuevo leon| 5229471|
|        campeche| 2611299|
| san luis potosi| 2605944|
|        veracruz| 4181993|
|         morelos| 1509095|
|      guanajuato| 4933886|
|          sonora| 3511149|
|        tlaxcala| 2843154|
|         nayarit|  992414|
|         sinaloa| 2264742|
|          oaxaca| 2244336|
|        guerrero| 1858948|
|    quintana roo| 4754708|
|       queretaro| 3385694|
|estado de mexico|17656040|
|          puebla| 3514964|
|         durango| 2389334|
|         jalisco| 6446410|
+----------------+--------+
only showing top 20 rows

- De cada estado obten: el número de catalogos diferentes por año, ¿ha aumentado el número de catálogos con el tiempo?

In [ ]:
from pyspark.sql.functions import year

# Count distinct catalogs per state per year
catalogs_per_state_per_year = df_parquet.groupBy("estado", "year").agg(countDistinct("catalogo").alias("distinct_catalogs"))
catalogs_per_state_per_year.show()

# To check if the number of catalogs has increased over the years, we will order the results
trend_catalogs = catalogs_per_state_per_year.orderBy("estado", "year")
trend_catalogs.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Utilizando Spark contesta las siguientes preguntas a partir **del catálogo que
le tocó a tu equipo**. Recuerda trabajar en el archivo con los datos particionados
de otra manera tus queries van a tardar mucho.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Material Escolar Analysis").getOrCreate()

# Read data from the specific category partition
df_cat = spark.read.parquet("s3://mdge-e3-2024/parquet_files_partitioned/categoria=material escolar/")

- ¿Cuańtas marcas diferentes tiene tu categoría?

In [ ]:
from pyspark.sql.functions import countDistinct

# Count distinct brands in the category
distinct_brands = df_cat.agg(countDistinct("marca").alias("distinct_brands"))
distinct_brands.show()

- ¿Cuál es la marca con mayor precio? ¿En qué estado?

In [ ]:
from pyspark.sql.functions import max, struct

# Find the brand with the highest price and the state
brand_highest_price = df_cat.select("marca", "estado", "precio").groupBy("marca", "estado").agg(max("precio").alias("max_precio"))
brand_highest_price.orderBy("max_precio", ascending=False).show(1)


- ¿Cuál es la marca con menor precio en CDMX? (en aquel entonces Distrito Federal)

In [ ]:
from pyspark.sql.functions import min

# Filter by CDMX (Distrito Federal) and find the brand with the lowest price
brand_lowest_price_cdmx = df_cat.filter(df_cat.estado == "ciudad de mexico").groupBy("marca").agg(min("precio").alias("min_precio"))
brand_lowest_price_cdmx.orderBy("min_precio", ascending=True).show(1)

- ¿Cuál es la marca con mayores observaciones?

In [ ]:
# Find the brand with the most observations
brand_most_observations = df_cat.groupBy("marca").count().orderBy("count", ascending=False)
brand_most_observations.show(1)

- ¿Cuáles son el top 5 de marcas con mayor precio en cada estado? ¿Son diferentes?

In [ ]:
# Find the top 5 brands with the highest price in each state
top5_brands_per_state = df_cat.groupBy("estado", "marca").agg(max("precio").alias("max_precio")).orderBy("estado", "max_precio", ascending=[True, False])
top5_brands_per_state.show()

- ¿Cuáles son el top 5 de marcas con menor precio en CDMX? (en aquel entonces Distrito Federal)

In [ ]:
# Filter by CDMX and find the top 5 brands with the lowest price
top5_lowest_price_cdmx = df_cat.filter(df_cat.estado == "ciudad de mexico").groupBy("marca").agg(min("precio").alias("min_precio")).orderBy("min_precio", ascending=True).limit(5)
top5_lowest_price_cdmx.show()


- ¿Cuáles son el top 5 de marcas con mayores observaciones? ¿Se parecen a las de nivel por estado?

In [ ]:
# Find the top 5 brands with the most observations globally
global_top5_observation_brands = df_cat.groupBy("marca").count().orderBy("count", ascending=False).limit(5)
global_top5_observation_brands.show()

- ¿Ha dejado de existir alguna marca durante los años que tienes? ¿Cuál? ¿Cuándo desapareció?

In [ ]:
from pyspark.sql.functions import min, max

# Find if any brand has disappeared over the years
# Assuming 'ano' is the year column in your DataFrame
brand_years = df_cat.groupBy("marca").agg(min("ano").alias("first_year"), max("ano").alias("last_year"))
current_year = df_cat.select(max("ano")).collect()[0][0]  # assuming this retrieves the current or latest year in the dataset
disappeared_brands = brand_years.filter(brand_years.last_year < current_year)
disappeared_brands.show()

- Genera una gráfica de serie de tiempo por estado para la marca con mayor precio -en todos los años-, donde el eje equis es el año y el eje ye es el precio máximo.

In [ ]:
# Extract data for visualization
brand_max_price_by_state_year = df_cat.groupBy("estado", "ano", "marca").agg(max("precio").alias("max_precio")).orderBy("estado